<a href="https://colab.research.google.com/github/JavierPerez21/QHack2022/blob/master/Coding_Challenges/games_500_switches_template/games_500_switches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install pennylane

In [ ]:
from pennylane import numpy as np
import pennylane as qml

This games consists of 3 switches and 1 lightbulb, each represented by a qubit that can be either on (1) or off (0). Some switches migh not work and to guess which work, we naively turn them on or off. They are set up in such a way that two working switches being activated cancel each other.

The goal of this challenge is to determine which switches work and which do with a single query to an oracle (as shown below) that represents the behaviour of the light with respect to the switches.


```
def oracle(switches):
    for i in switches:
        qml.CNOT(wires=[i, "light"])
```

This problem can be though of as an application of the [Bernstein-Vazirani algorithm](https://qiskit.org/textbook/ch-algorithms/bernstein-vazirani.html) in which the state $|s\rangle$ we want to find represents the working of the switches. 

In [ ]:
def switch(oracle):
    """Function that, given an oracle, returns a list of switches that work by executing a
    single circuit with a single shot. The code you write for this challenge should be completely
    contained within this function between the # QHACK # comment markers.

    Args:
        - oracle (function): oracle that simulates the behavior of the lights.

    Returns:
        - (list(int)): List with the switches that work. Example: [0,2].
    """

    dev = qml.device("default.qubit", wires=[0, 1, 2, "light"], shots=1)

    @qml.qnode(dev)
    def circuit():

        # QHACK #
        # Bernstein-Vazirani preprocessing
        for i in range(3):
            qml.Hadamard(wires=i)
        qml.PauliX(wires="light")
        qml.Hadamard(wires="light")
        # Oracle
        oracle()
        # Bernstein-Vazirani postprocessing
        for i in range(3):
            qml.Hadamard(wires=i)
        # QHACK #

        return qml.sample(wires=range(3))

    sample = circuit()
    
    # QHACK #
    # Getnumber of switches that work
    sample = [i for i in range(0, len(sample)) if sample[i] == 1]
    # QHACK #
    return sample

Testing with 1.in

In [ ]:
inputs = [0,1,2,0]
# By this convention, if the switch index appears an even number of times, it will not work. If the switch appears an odd number of times, 
# the switch will work.

numbers = [int(i) for i in inputs]

def oracle():
    for i in numbers:
        qml.CNOT(wires=[i, "light"])

output = switch(oracle)
print("The switches that work are", output)